### Experiments to find the best evolutionary parameters to optimize the impact of local nudges

In [ ]:
import json
import numpy as np
from scipy import stats

import probability_distributions
import maximum_nudges
import evolutionary_algorithms as ea
import maximum_nudges_evolutionary
import get_data
import nudge_non_causal

In [ ]:
NUDGE_SIZE = 0.01
PATH_TO_DISTRIBUTIONS = "/home/joboti/azumi_derkjan/master_thesis/code/"

#### Define the input and output distributions for local nudges

In [ ]:
PERCENTAGE_MAX_ENTROPY_SIZE = 75
NUMBER_OF_VARS = 4
NUMBER_OF_STATES = 3
FILENAME_FORMAT_INPUT = "input_dist_{}.npy"
FILENAME_FORMAT_OUTPUT = "cond_output_dist_{}.npy"
DIST_START = 100
DIST_END = 103

distribution_numbers = list(range(DIST_START, DIST_END, 1))
path_to_limited_entropy_system_dists = (
    PATH_TO_DISTRIBUTIONS + "system_distributions/" 
    + "limited_entropy/"
)
input_dists = get_data.get_system_distributions_limited_entropy(
    path_to_limited_entropy_system_dists, PERCENTAGE_MAX_ENTROPY_SIZE,
    NUMBER_OF_VARS, NUMBER_OF_STATES, FILENAME_FORMAT_INPUT, 
    distribution_numbers
)
input_dists = [input_dist for input_dist in input_dists]
output_dists = get_data.get_system_distributions_limited_entropy(
    path_to_limited_entropy_system_dists, PERCENTAGE_MAX_ENTROPY_SIZE,
    NUMBER_OF_VARS, NUMBER_OF_STATES, FILENAME_FORMAT_OUTPUT, 
    distribution_numbers
)
output_dists = [output_dist for output_dist in output_dists]

#### Define the evolutionary parameters

In [ ]:
evolutionary_params = {
    "number_of_generations": 50,
    "population_size": 20,
    "number_of_children": 150,
    "generational": False,
    "parent_selection_mode": "rank_exponential",
    "mutation_size_weights": 0.8,
    "change_mutation_size_weights": 0.5,
    "start_mutation_size": 0.005,
    "change_mutation_size": 0.5
}


In [ ]:
maximum_nudges_evolutionary.TEST = False
impacts = []
backup_filename = "find_params_local_nudge_{}var_{}states_{}entropy_dists{}_{}.json".format(
    NUMBER_OF_VARS, NUMBER_OF_STATES, PERCENTAGE_MAX_ENTROPY_SIZE, DIST_START, DIST_END 
)
for input_dist, cond_output in zip(input_dists, output_dists):
    max_local_nudge = maximum_nudges_evolutionary.find_maximum_local_nudge(
        input_dist, cond_output, NUDGE_SIZE, evolutionary_params
    )
    impact = max_local_nudge.score
    individual_nudges = max_local_nudge.individual_nudges
    nudge_vectors = [
        weight*nudge.genes 
        for nudge, weight in zip(individual_nudges, max_local_nudge.weights)
    ]
    new_dist = nudge_non_causal.nudge_local(
        input_dist, list(range(NUMBER_OF_VARS-1)), NUDGE_SIZE, nudge_vectors 
    )
    l1_norm_to_old_distribution = np.sum(np.absolute(input_dist-new_dist))
    if abs(l1_norm_to_old_distribution-NUDGE_SIZE) > 10**(-5):
        print("WARNING size of nudge {}".format(l1_norm_to_old_distribution))

    print("the max nudge impact {}".format(impact))
    impacts.append(impact)
    with open(backup_filename, 'w') as f:
        json.dump(impacts, f, indent=4)

print(np.mean(impacts))
print(np.std(impacts))
    